In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

% matplotlib inline

# Question 1: Propensity score matching

# Question 2: Applied ML

## 2.1 Data loading and features extraction

First we will download the dataset as explained and take a look at the dataset. It is a collection of approximately 20,000 newsgroup documents, partitioned across 20 different newsgroups. For more information go to [this website](http://qwone.com/~jason/20Newsgroups/). The data is composed of multiple fields.

* `data`: is the actual text and description of the news.
* `target`: is the label (as integers) of the news that are part of the 20 categories present in `target_names` (displayed later).
* `filename`: is the location of the file.
* `descritpion`: is the title of the dataset, here : "the 20 newsgroups by date dataset".
* `DESCR` is empty
* `target_names`: is the list of target we are trying to match (news categories linked to `target`).

In [ ]:
newsgroups = fetch_20newsgroups(subset='all')

In [ ]:
list(newsgroups.keys())

Note that some labels are actually sub categroies. For example `politics.guns` and `politics.mideast` are both subcategories of `talk`. We can expect it will be more difficult to discriminate and classify news that are part of the same category.

In [ ]:
newsgroups.target_names

We can now create our TF-IDF matrix. The matrix will be sparse and huge since we are not removing any words or limiting the number of features. Here for example the size of vocabulary (features) is 173762.

In [ ]:
tfidf = TfidfVectorizer()
x_tfidf = tfidf.fit_transform(newsgroups.data) 
len(tfidf.vocabulary_)

As asked we will split our data in 3 different sets: `train` (80%), `validation` (10%) and `test` (10%). We will assert our results on the validation set before testing it on the test set. Note that we fixed the seed to 0 for reproducibility purposes.

In [ ]:
ratio_train = 0.8
ratio_validation = 0.1

np.random.seed(0)
id_ = np.random.permutation(np.arange(x_tfidf.shape[0]))
id_train = id_[:np.ceil(x_tfidf.shape[0]*ratio_train).astype(int)]
id_validation = id_[np.floor(x_tfidf.shape[0]*ratio_train).astype(int):
                    np.ceil(x_tfidf.shape[0]*(ratio_train+ratio_validation)).astype(int)]
id_test = id_[np.floor(x_tfidf.shape[0]*(ratio_train+ratio_validation)).astype(int):]

Now that we randomly split our data ids we can create our features sets and labels for classification.

In [ ]:
x_train = x_tfidf[id_train]
x_validation = x_tfidf[id_validation]
x_test = x_tfidf[id_test]

y_train = newsgroups.target[id_train]
y_validation = newsgroups.target[id_validation]
y_test = newsgroups.target[id_test]

## 2.2 Classification using Random Forest

### 2.2.1 Train and Validation set

We decided to go from 0 to around 200 features for both `max_depth` and `n_estimators`. Note that we choosed a logaritmic scale for features numbers since there are no resons to compare values such as for example `max_depth`=63 to `max_depth`=64 since they will give similar results.

In [ ]:
span_depth = np.ceil(np.logspace(0, 2.3, 15)).astype(int)
span_estimators = np.ceil(np.logspace(0, 2.3, 15)).astype(int)

In [ ]:
score = np.zeros((span_depth.shape[0], span_estimators.shape[0]))

for i, n_depth in enumerate(span_depth):
    for j, n_estimator in enumerate(span_estimators):
        clf = RandomForestClassifier(max_depth=n_depth, n_estimators=n_estimator, random_state=0)
        clf.fit(x_train, y_train)
        y_pred = clf.predict(x_validation)
        score[i, j] = accuracy_score(y_pred, y_validation)
    print('Step {}/{} - Best accuracy max_depth={} is {:.4f}'.format(
        i+1, len(span_depth), n_depth, np.max(score[i])))

Since the grid search takes some time to run we saved the results to save time for next runs. Of course you can run the code from scratch at any time to test the results.

In [ ]:
np.save('score_random_forest.npy', 
        {'max_depth': span_depth, 'n_estimators': span_estimators, 'score': score})

In [ ]:
data = np.load('score_random_forest.npy')[()]

We can take a look at the accuracies and we observe that the larger the parameters `n_estimators` and `max_depth` are, the better the accuracy is. However we can also observe that we are reaching some limit. It seems that the algorithm is reaching a plateau around 86%.

In [ ]:
df_score = pd.DataFrame(data['score'])
df_score.columns = [str(num) for num in data['n_estimators']]
df_score.index = [str(num) for num in data['max_depth']]

plt.figure(figsize=(10, 8))
ax = sns.heatmap(df_score, annot=True, fmt=".2f", cmap='RdYlGn')
ax.set_xlabel('n_estimators'); ax.set_ylabel('max_depth');
ax.set_title('Accuracy score over "Max Depth" and "N Estimators"')

Here we can even more clearly see that we reached a plateau. There is only 4% augmentation in accuray for a difference of 169 in # of Estimators. As a consequence, we choosed to not try with higher values since they will not represent a significative improvement in accuracy.

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(df_score.index, df_score.iloc[-1], label='max_d=200')
plt.plot(df_score.index, df_score.iloc[-3], label='max_d=94')
plt.plot(df_score.index, df_score.iloc[-6], label='max_d=31')
plt.xlabel('# Estimators'); plt.ylabel('Accuracy in %'); plt.grid(); 
plt.title('Accuracy on validation set as a funtion of MaxDepth and #Estimators'); plt.legend()

### 2.2.2 Results on Test set

We used the values we best score on validation set e.i. (Max Depth, # Estimators) = (200, 200). We can see that the result on the test set is similar to the one on the validation set with around 86% in accuracy. 

In [ ]:
clf = RandomForestClassifier(max_depth=200, n_estimators=200, random_state=0)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print('Accuracy on train set is {:.4f}'.format(accuracy_score(y_pred, y_test)))

To look more carefuly at the results we can compute the confusion matrix. Note that, by default, the matrix is not normalized and it is difficult to look at coherence of the results. We choosed to normalize it with 

$$\text{precision}_j = \frac{\text{tp}_j}{\text{tp}_j + \sum_{i \neq j} \text{fp}_i} $$

where $\text{tp}_j$ is the amount of true positive and $\text{fp}_i$ are the number of false positive for each class $j$. Note that we did not consider recall in this case.

In [ ]:
cm = confusion_matrix(y_pred, y_test)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

In [ ]:
df_conf = pd.DataFrame(cm)
df_conf.columns = newsgroups.target_names
df_conf.index = newsgroups.target_names

We can see that most classes are performing well with values close to 1. However we can see that some classes such as `soc.religion.christian` only have around 0.7 precision. It is interesting to notice that most of the fp are acually part of `talk.religion.misc` and fewer are `alt.atheism`. Those two topics are indeed related to the first one and we can conclude classification is harder. It is also interesting to notice that `misc.forsale` have fp spread around other classes such as `sci.electronics`, `comp.sys.mac.hardware`, or even `comp.graphics`. Thoses make also sense since this category (forsale) can include a lot of electronic devices and miscellaneous objects.

In [ ]:
plt.figure(figsize=(14, 10))
ax = sns.heatmap(df_conf, annot=True, fmt=".2f", cmap='RdYlGn')

### 2.2.3 Feature Importances
Let's now look at the features more carefuly. If we plot the repartition of the feature importances we can see that there are actually a lot of features that have little importance in classification (close to 0). Moreover there are only around 20 feature that are above 0.002. Note that the y scale is logaritmic. 

In [ ]:
plt.figure(figsize=(10,5))
plt.hist(clf.feature_importances_, bins=50)
plt.yscale('log', nonposy='clip')
plt.xlabel('Feature importance'); plt.ylabel('#Features');
plt.grid(); plt.title('Repartition of feature importance')

As explained before it seems that a small set of top feature have huge impact on clssification. Let's display the 50 with highest importance and therefore look at the relevant words use for classification. It is interessting to notice that some words are actually part of the categories (labels) names. For example "baseball" with `rec.sport.baseball`, "gun" with `talk.politics.gun` or "sale" with `forsale`. However it is wierd to have some unrelevant words such as "are", "you", "for", "the" or even "it" that are often considered as stop words.

In [ ]:
n_features = 50
id_max = np.argsort(clf.feature_importances_)[-n_features:]
np.array(tfidf.get_feature_names())[id_max]